# 1. Ingeniería de Características y Creación de Secuencias LSTM

Este notebook toma el dataset preprocesado (`df_final` guardado desde el Notebook 1) y lo transforma en el formato de secuencias necesario para entrenar un modelo LSTM.

**Pasos Realizados:**

1.  **Carga de Datos:** Se carga el `df_final` (en formato `.csv` o `.parquet`).
2.  **Preprocesamiento Mínimo:**
    * **Ordenamiento:** Se asegura de que los datos estén estrictamente ordenados cronológicamente por cliente (`POLIZA_SUMINISTRO`), `FECHA` y `HORA`. **Esto es crucial** para la correcta creación de secuencias temporales.
    * **Verificación de Tipos:** Realiza una comprobación final y conversión (si es necesario) de las columnas numéricas (clima, consumo) a `float`, la columna `FESTIVO` a `int`, y aplica *one-hot encoding* a `TIPO_DIA` si no se hizo previamente.
3.  **Definición de Features y Target:**
    * Se definen las columnas que actuarán como **features (`X`)** para el modelo. Estas incluyen `CONSUMO_REAL`, las variables climáticas (`TEMP_MEDIA`, `TEMP_MIN`, `TEMP_MAX`, `PRECIPITACION`, `HUMEDAD_RELATIVA_MEDIA`), `FESTIVO`, y las columnas generadas por el *one-hot encoding* de `TIPO_DIA`. Se excluyen IDs y columnas de fecha/hora.
    * Se define la columna objetivo (`y`) como `FUGA_DETECTADA`.
4.  **Manejo Final de NaNs:** Se realiza una última verificación de valores nulos en las *features* y el *target*. Si se encuentran, se aplica un relleno (`ffill` y `bfill`) **agrupado por cliente** para evitar rellenar con datos de otros clientes. Las filas con NaNs persistentes se eliminan.
5.  **Creación de Secuencias (Ventana Deslizante):**
    * Se agrupan los datos por `POLIZA_SUMINISTRO`.
    * Para cada cliente, se desliza una "ventana" de tamaño `N_PAST_STEPS` (ej. 72 horas/registros) a través de sus datos.
    * Cada ventana de *features* se convierte en una muestra para el array `X`.
    * El valor de `FUGA_DETECTADA` inmediatamente posterior a cada ventana se toma como el objetivo para el array `y`.
    * Este proceso garantiza que las secuencias solo contengan datos de un único cliente y en orden cronológico.
6.  **Guardado de Datos:**
    * Los arrays resultantes `X` (forma: `[número_de_secuencias, N_PAST_STEPS, número_de_features]`) e `y` (forma: `[número_de_secuencias]`) se guardan como archivos `.npy` **sin escalar**.
    * **Importante:** El escalado de *features* (ej., `MinMaxScaler`) se realizará en el siguiente notebook (`03_Model_Training.ipynb`) **después** de dividir los datos en conjuntos de entrenamiento, validación y prueba para evitar fuga de datos (*data leakage*).

In [30]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm 
import os 

## 1.1. Carga de datos

In [31]:
# Load final_dataset
df_final = pd.read_csv('../data/final_dataset.csv')

## 1.2. Preprocesamiento mínimo

In [32]:
# Minimal preprocessing to ensure correct types and sorting
print("\n--- Applying Minimal Preprocessing ---")

# Ensure correct sorting 
# Convert 'HORA' to string if not already
df_final['HORA'] = df_final['HORA'].astype(str)
df_final = df_final.sort_values(by=['POLIZA_SUMINISTRO', 'FECHA', 'HORA'])
print("Sorted DataFrame by POLIZA_SUMINISTRO, FECHA, and HORA.")

# Convert numeric columns from object to float if necessary
numeric_cols = ['TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 'HUMEDAD_RELATIVA_MEDIA', 'CONSUMO_REAL']
for col in numeric_cols:
    if col in df_final.columns and df_final[col].dtype == 'object':
        print(f"Converting '{col}' back to numeric (cleaning just in case)...")
        df_final[col] = df_final[col].astype(str).str.replace(',', '.', regex=False)
        df_final[col] = df_final[col].str.extract(r'([+-]?\d+(?:\.\d*)?|[+-]?\.\d+)', expand=False)
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce')
    elif col in df_final.columns:
         # Ensure it's float
         df_final[col] = pd.to_numeric(df_final[col], errors='coerce').astype(float)


if 'FESTIVO' in df_final.columns and df_final['FESTIVO'].dtype != 'int':
    df_final['FESTIVO'] = df_final['FESTIVO'].astype(int)
    print("Converted 'FESTIVO' to integer.")

# One-hot encode 'TIPO_DIA' if it exists and is categorical
initial_cols = set(df_final.columns)
new_one_hot_cols = []
if 'TIPO_DIA' in df_final.columns and df_final['TIPO_DIA'].dtype == 'object':
    print("One-hot encoding 'TIPO_DIA'...")
    df_final['TIPO_DIA'] = df_final['TIPO_DIA'].astype('category')
    df_final = pd.get_dummies(df_final, columns=['TIPO_DIA'], drop_first=True, dummy_na=False)
    new_one_hot_cols = list(set(df_final.columns) - initial_cols)
    print(f"Encoded 'TIPO_DIA' into: {new_one_hot_cols}")
else:
    # Check if already encoded
    new_one_hot_cols = [col for col in df_final.columns if col.startswith('TIPO_DIA_')]
    if new_one_hot_cols:
        print(f"'TIPO_DIA' seems already encoded into: {new_one_hot_cols}")
    else:
        print("'TIPO_DIA' column not found or already numeric.")


--- Applying Minimal Preprocessing ---
Sorted DataFrame by POLIZA_SUMINISTRO, FECHA, and HORA.
Converting 'TEMP_MEDIA' back to numeric (cleaning just in case)...
Converting 'TEMP_MIN' back to numeric (cleaning just in case)...
Converting 'TEMP_MAX' back to numeric (cleaning just in case)...
Converting 'PRECIPITACION' back to numeric (cleaning just in case)...
Converted 'FESTIVO' to integer.
One-hot encoding 'TIPO_DIA'...
Encoded 'TIPO_DIA' into: ['TIPO_DIA_Laborable', 'TIPO_DIA_Fin de Semana']


## 1.3. Definición de Features y Target

In [33]:
# Define Features and Target 

N_PAST_STEPS = 72 # Number of past time steps (rows) to use as input
N_FUTURE_STEPS = 1 # Number of future time steps to predict (typically 1 for classification)

# Define feature columns
FEATURE_COLUMNS = [
    'CONSUMO_REAL',
    'TEMP_MEDIA',
    'TEMP_MIN',
    'TEMP_MAX',
    'PRECIPITACION',
    'HUMEDAD_RELATIVA_MEDIA',
    'FESTIVO',
] + new_one_hot_cols 

# Filter FEATURE_COLUMNS to only those actually present in the DataFrame
FEATURE_COLUMNS = [col for col in FEATURE_COLUMNS if col in df_final.columns]
print(f"\nUsing features: {FEATURE_COLUMNS}")


TARGET_COLUMN = 'FUGA_DETECTADA'
if TARGET_COLUMN not in df_final.columns:
     print(f"ERROR: Target column '{TARGET_COLUMN}' not found.")
     exit()
     
# Ensure target column is integer
df_final[TARGET_COLUMN] = df_final[TARGET_COLUMN].astype(int)



Using features: ['CONSUMO_REAL', 'TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 'HUMEDAD_RELATIVA_MEDIA', 'FESTIVO', 'TIPO_DIA_Laborable', 'TIPO_DIA_Fin de Semana']


## 1.4. Manejo final de NaNs (Doble verificación)

In [34]:
# handle NaNs in features and target
print("\nFinal check for NaNs before creating sequences:")
print(df_final[FEATURE_COLUMNS + [TARGET_COLUMN]].isnull().sum())
if df_final[FEATURE_COLUMNS + [TARGET_COLUMN]].isnull().values.any():
    print("Applying final ffill and bfill for NaNs...")
    # Group by customer is essential here
    df_final[FEATURE_COLUMNS] = df_final.groupby('POLIZA_SUMINISTRO')[FEATURE_COLUMNS].ffill()
    df_final[FEATURE_COLUMNS] = df_final.groupby('POLIZA_SUMINISTRO')[FEATURE_COLUMNS].bfill()
    initial_rows = len(df_final)
    # Drop rows where NaNs couldn't be filled (e.g., beginning of a customer's data)
    df_final.dropna(subset=FEATURE_COLUMNS + [TARGET_COLUMN], inplace=True)
    if len(df_final) < initial_rows:
        print(f"Dropped {initial_rows - len(df_final)} rows with persistent NaNs.")
print("Final NaN handling complete.")


Final check for NaNs before creating sequences:
CONSUMO_REAL              0
TEMP_MEDIA                0
TEMP_MIN                  0
TEMP_MAX                  0
PRECIPITACION             0
HUMEDAD_RELATIVA_MEDIA    0
FESTIVO                   0
TIPO_DIA_Laborable        0
TIPO_DIA_Fin de Semana    0
FUGA_DETECTADA            0
dtype: int64
Final NaN handling complete.


## 1.5. Creación de secuencias (ventana deslizante)

In [35]:
# Implement Sliding Window per Customer 
print("\n--- Implementing Sliding Window ---")

X_sequences = []
y_targets = []

grouped = df_final.groupby('POLIZA_SUMINISTRO')
print(f"Creating sequences with N_PAST_STEPS={N_PAST_STEPS}...")

for customer_id, group in tqdm(grouped):
    # Select feature columns and target column as NumPy arrays
    features = group[FEATURE_COLUMNS].values
    target = group[TARGET_COLUMN].values

    # Ensure there's enough data for at least one sequence + target
    if len(group) >= N_PAST_STEPS + N_FUTURE_STEPS:
        # Iterate to create sequences
        for i in range(len(group) - N_PAST_STEPS - N_FUTURE_STEPS + 1):
            X_sequences.append(features[i : i + N_PAST_STEPS])
            # Target is the value N_FUTURE_STEPS after the sequence ends
            y_targets.append(target[i + N_PAST_STEPS + N_FUTURE_STEPS - 1]) # Adjust index based on N_FUTURE_STEPS

# Convert lists to NumPy arrays
X = np.array(X_sequences)
y = np.array(y_targets)

print(f"\nGenerated sequences:")
if X.size > 0 and y.size > 0:
    print(f"X shape: {X.shape}") # (num_samples, N_PAST_STEPS, num_features)
    print(f"y shape: {y.shape}") # (num_samples,)
else:
    print("WARNING: No sequences were generated. Check data length and parameters.")
    exit() # Exit if no sequences could be created


--- Implementing Sliding Window ---
Creating sequences with N_PAST_STEPS=72...


100%|██████████| 10/10 [00:00<00:00, 68.38it/s]



Generated sequences:
X shape: (121114, 72, 9)
y shape: (121114,)


## 1.6. Guardar datos de secuencias

In [36]:
# Save UN-SCALED Data into data directory
print("\n--- Saving Prepared Data ---")

# Define the target directory relative to the notebook's location
data_folder = '../data/'

# Create the directory if it doesn't exist (optional but good practice)
os.makedirs(data_folder, exist_ok=True)

# Define full file paths using os.path.join
x_filename = os.path.join(data_folder, 'X_sequences_unscaled.npy')
y_filename = os.path.join(data_folder, 'y_targets.npy')

try:
    np.save(x_filename, X)
    np.save(y_filename, y)
    print(f"\nUNSCALED prepared data saved successfully:")
    print(f"- Sequences saved to: {x_filename}")
    print(f"- Targets saved to: {y_filename}")
except Exception as e:
    print(f"\nERROR saving prepared data: {e}")


--- Saving Prepared Data ---

UNSCALED prepared data saved successfully:
- Sequences saved to: ../data/X_sequences_unscaled.npy
- Targets saved to: ../data/y_targets.npy
